<a href="https://colab.research.google.com/github/niekh-13/geodata-etl-workshop/blob/main/Introductie_GeoPandas_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introductie Pandas en GeoPandas: eenvoudige ETL scripting

In deze workshop voor Kaartviewer inspiratie dagen 2024 leer je de basis van Pandas en GeoPandas kennen door gebruik te maken van open data van Nederlandse netbeheerders en CBS.

In [ ]:
%%capture
# Install necessary packages
!pip install pandas geopandas ipython-extensions shapely

In [ ]:
%%capture

import requests
import zipfile
import os

## Stap 1: Data downloaden van gekozen netbeheerder

### Kies één van de netbeheerders en download hun dataset.


#### Liander



In [ ]:
# Download Liander
url = "https://www.liander.nl/-/media/files/open-data/kleinverbruikdata/kleinverbruiksgegevens-2024.zip"
response = requests.get(url)
delimiter = ';'

# Get filenames and paths for Liander
filename = 'Liander_kleinverbruiksgegevens_20240101.csv'
zip_path = url.split("/")[-1]

# Write response content in to zipfile
with open(zip_path, "wb") as f:
  f.write(response.content)

# Extract the csv file from zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(".")

print(f"{filename} is downloaded")

Liander_kleinverbruiksgegevens_20240101.csv is downloaded


#### Enexis


In [ ]:
# Download Enexis
url = "https://enxp433-oda01.s3.eu-west-1.amazonaws.com/kv/Enexis_kleinverbruiksgegevens_01012024.csv"
response = requests.get(url)
delimiter = ';'

# Get filename for Enexis
filename = url.split("/")[-1]

# Write response content in to csv file
with open(filename, "wb") as f:
  f.write(response.content)

print(f"{filename} is downloaded")


Enexis_kleinverbruiksgegevens_01012024.csv is downloaded


#### Stedin

In [ ]:
!wget https://www.stedin.net/-/media/project/online/files/zakelijk/open-data/stedin-kleinverbruikgegevens-2024.csv

delimiter = '\t'

--2024-09-13 12:28:04--  https://www.stedin.net/-/media/project/online/files/zakelijk/open-data/stedin-kleinverbruikgegevens-2024.csv
Resolving www.stedin.net (www.stedin.net)... 20.101.166.0
Connecting to www.stedin.net (www.stedin.net)|20.101.166.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23869452 (23M) [text/csv]
Saving to: ‘stedin-kleinverbruikgegevens-2024.csv’

stedin-kleinverbrui 100%[===================>]  22.76M  16.9MB/s    in 1.3s    

2024-09-13 12:28:06 (16.9 MB/s) - ‘stedin-kleinverbruikgegevens-2024.csv’ saved [23869452/23869452]



#### Coteq

In [ ]:
# Download Coteq
url = "https://d3a07q56iliqjn.cloudfront.net/web-uploads/Documenten/Open-data/CoteqNetbeheer_kleinverbruik_01012024.csv"
response = requests.get(url)
delimiter = ';'

# Get filename for Coteq
filename = url.split("/")[-1]

# Write response content in to csv file
with open(filename, "wb") as f:
  f.write(response.content)

print(f"{filename} is downloaded")

CoteqNetbeheer_kleinverbruik_01012024.csv is downloaded



## Stap 2: Netbeheer data in pandas laden

In [ ]:
import pandas as pd

columns = [
    "NETBEHEERDER", "NETGEBIED", "STRAATNAAM", "POSTCODE_VAN", "POSTCODE_TOT",
    "WOONPLAATS", "LANDCODE", "PRODUCTSOORT", "VERBRUIKSSEGMENT", "AANSLUITINGEN_AANTAL",
    "LEVERINGSRICHTING_PERC", "FYSIEKE_STATUS_PERC", "SOORT_AANSLUITING_PERC",
    "SOORT_AANSLUITING", "SJV_GEMIDDELD", "SJV_LAAG_TARIEF_PERC", "SLIMME_METER_PERC"
]

# Inlezen van netbeheerder data met pandas
data = pd.read_csv(filename, sep=delimiter, dtype=str, names=columns, skiprows=1)

# Data van netbeheerder uniform maken voor pandas
data = data.map(lambda x: x.replace(',', '.') if isinstance(x, str) else x)

#### Controleer data

In [ ]:
print(data.head())
# print(data.info())
# print(data.describe())

  NETBEHEERDER NETGEBIED STRAATNAAM POSTCODE_VAN POSTCODE_TOT WOONPLAATS  \
0  Enexis B.V.    ENEXIS    Sasdijk      4251 AA      4251 AA  WERKENDAM   
1  Enexis B.V.    ENEXIS    Sasdijk      4251 AA      4251 AA  WERKENDAM   
2  Enexis B.V.    ENEXIS    Sasdijk      4251 AB      4251 AB  WERKENDAM   
3  Enexis B.V.    ENEXIS    Sasdijk      4251 AB      4251 AB  WERKENDAM   
4  Enexis B.V.    ENEXIS    Sasdijk      4251 AC      4251 AC  WERKENDAM   

  LANDCODE PRODUCTSOORT VERBRUIKSSEGMENT AANSLUITINGEN_AANTAL  \
0       NL          ELK              KVB                   41   
1       NL          GAS              KVB                   40   
2       NL          ELK              KVB                   15   
3       NL          GAS              KVB                   14   
4       NL          ELK              KVB                   11   

  LEVERINGSRICHTING_PERC FYSIEKE_STATUS_PERC SOORT_AANSLUITING_PERC  \
0                  87.80                 100                  51.22   
1         

#### Data aggereren naar Postcode4

In [ ]:
# Stap 1: Postcode4 afleiden
data['POSTCODE4'] = data['POSTCODE_TOT'].str[:4]

# Zorg dat AANSLUITINGEN_AANTAL en andere numerieke kolommen numeriek zijn
data['POSTCODE4'] = pd.to_numeric(data['POSTCODE4'], errors='coerce')
data['AANSLUITINGEN_AANTAL'] = pd.to_numeric(data['AANSLUITINGEN_AANTAL'], errors='coerce')
data['SJV_GEMIDDELD'] = pd.to_numeric(data['SJV_GEMIDDELD'], errors='coerce')
data['SJV_LAAG_TARIEF_PERC'] = pd.to_numeric(data['SJV_LAAG_TARIEF_PERC'], errors='coerce')
data['LEVERINGSRICHTING_PERC'] = pd.to_numeric(data['LEVERINGSRICHTING_PERC'], errors='coerce')
data['FYSIEKE_STATUS_PERC'] = pd.to_numeric(data['FYSIEKE_STATUS_PERC'], errors='coerce')
data['SOORT_AANSLUITING_PERC'] = pd.to_numeric(data['SOORT_AANSLUITING_PERC'], errors='coerce')
data['SLIMME_METER_PERC'] = pd.to_numeric(data['SLIMME_METER_PERC'], errors='coerce')

# Stap 2: Functie om gewogen gemiddelde te berekenen
def weighted_average(df, col, weight_col):
    return (df[col] * df[weight_col]).sum() / df[weight_col].sum()

# Stap 3: Groeperen op de gewenste kolommen
grouped_data = data.groupby(
    ['POSTCODE4', 'PRODUCTSOORT', 'NETBEHEERDER', 'NETGEBIED', 'WOONPLAATS', 'LANDCODE', 'VERBRUIKSSEGMENT']
).apply(lambda x: pd.Series({
    'AANSLUITINGEN_TOTAAL': x['AANSLUITINGEN_AANTAL'].sum(),
    'SJV_GEMIDDELD_GEM': weighted_average(x, 'SJV_GEMIDDELD', 'AANSLUITINGEN_AANTAL'),
    'SJV_LAAG_TARIEF_PERC_GEM': weighted_average(x, 'SJV_LAAG_TARIEF_PERC', 'AANSLUITINGEN_AANTAL'),
    'LEVERINGSRICHTING_PERC_GEM': weighted_average(x, 'LEVERINGSRICHTING_PERC', 'AANSLUITINGEN_AANTAL'),
    'FYSIEKE_STATUS_PERC_GEM': weighted_average(x, 'FYSIEKE_STATUS_PERC', 'AANSLUITINGEN_AANTAL'),
    'SOORT_AANSLUITING_PERC_GEM': weighted_average(x, 'SOORT_AANSLUITING_PERC', 'AANSLUITINGEN_AANTAL'),
    'SLIMME_METER_PERC_GEM': weighted_average(x, 'SLIMME_METER_PERC', 'AANSLUITINGEN_AANTAL'),
})).reset_index()

In [ ]:
# Stap 4: Bekijk de gegroepeerde data
# print(grouped_data.head())
print(grouped_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3963 entries, 0 to 3962
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   POSTCODE4                   3963 non-null   int64  
 1   PRODUCTSOORT                3963 non-null   object 
 2   NETBEHEERDER                3963 non-null   object 
 3   NETGEBIED                   3963 non-null   object 
 4   WOONPLAATS                  3963 non-null   object 
 5   LANDCODE                    3963 non-null   object 
 6   VERBRUIKSSEGMENT            3963 non-null   object 
 7   AANSLUITINGEN_TOTAAL        3963 non-null   float64
 8   SJV_GEMIDDELD_GEM           3963 non-null   float64
 9   SJV_LAAG_TARIEF_PERC_GEM    3963 non-null   float64
 10  LEVERINGSRICHTING_PERC_GEM  3963 non-null   float64
 11  FYSIEKE_STATUS_PERC_GEM     3963 non-null   float64
 12  SOORT_AANSLUITING_PERC_GEM  3963 non-null   float64
 13  SLIMME_METER_PERC_GEM       3963 

## Stap 3: CBS Postcode Data downloaden

In [ ]:
# Make variables for download
pc4_url = "https://download.cbs.nl/postcode/2024-cbs_pc4_2023_v1.zip"
pc4_dirname = "CBS_Postcode" # Name of the directory

# Download CBS Postcode data
response = requests.get(pc4_url)

# Get filename for CBS Postcode
filename = pc4_url.split("/")[-1]

# Write response content in to zip file
with open(filename, "wb") as f:
    f.write(response.content)

# Extract the files from zip file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(f"./{pc4_dirname}")

print(f"{pc4_dirname} data is gedownload en uitgepakt")

CBS_Postcode data is gedownload en uitgepakt


## Stap 4: CBS Postcode data inlezen

In [ ]:
import geopandas as gpd

In [ ]:
# Bestandspad naar het GeoPackage-bestand
cbs_postcode_file = "CBS_Postcode/cbs_pc4_2023_v1.gpkg"

# CBS Postcode data inlezen
cbs_postcode = gpd.read_file(cbs_postcode_file, layer='cbs_pc4_2023')

In [ ]:
# Alleen de kolommen 'postcode6' en 'geometry' selecteren
cbs_postcode = cbs_postcode[['postcode', 'geometry']]

#### Controleer data

In [ ]:
# print(cbs_postcode.head())
print(cbs_postcode.info())
# print(cbs_postcode.describe())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4070 entries, 0 to 4069
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   postcode  4070 non-null   int64   
 1   geometry  4070 non-null   geometry
dtypes: geometry(1), int64(1)
memory usage: 63.7 KB
None


## Stap 5: CBS Postcode Data koppelen aan netbeheer data


In [ ]:
# Merging the datasets
merged_data = pd.merge(grouped_data, cbs_postcode, left_on="POSTCODE4", right_on="postcode", how="left")
merged_data = merged_data.drop(columns=["postcode"])
merged_data = gpd.GeoDataFrame(merged_data, geometry='geometry')

#### Controleer data

In [ ]:
# print(merged_data.head())
print(merged_data.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3963 entries, 0 to 3962
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   POSTCODE4                   3963 non-null   int64   
 1   PRODUCTSOORT                3963 non-null   object  
 2   NETBEHEERDER                3963 non-null   object  
 3   NETGEBIED                   3963 non-null   object  
 4   WOONPLAATS                  3963 non-null   object  
 5   LANDCODE                    3963 non-null   object  
 6   VERBRUIKSSEGMENT            3963 non-null   object  
 7   AANSLUITINGEN_TOTAAL        3963 non-null   float64 
 8   SJV_GEMIDDELD_GEM           3963 non-null   float64 
 9   SJV_LAAG_TARIEF_PERC_GEM    3963 non-null   float64 
 10  LEVERINGSRICHTING_PERC_GEM  3963 non-null   float64 
 11  FYSIEKE_STATUS_PERC_GEM     3963 non-null   float64 
 12  SOORT_AANSLUITING_PERC_GEM  3963 non-null   float64 
 13  SLIMME_MET

## Stap 6: CBS wijkbuurten kaart data downloaden voor gemeenten

In [ ]:
# Make variables for download
cbv_url = "https://download.cbs.nl/regionale-kaarten/wijkbuurtkaart_2023_v1.zip"
cbs_name = "CBS_wijkbuurtkaart" # Name of the directory

# Download CBS Wijken data
response = requests.get(cbv_url)

# Get filename for Enexis
filename = cbv_url.split("/")[-1]

# Write response content in to zip file
with open(filename, "wb") as f:
    f.write(response.content)

# Extract the files from zip file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(f"./{cbs_name}")

print(f"{cbs_name} data is gedownload en uitgepakt")

CBS_wijkbuurtkaart data is gedownload en uitgepakt


In [ ]:
# CBS Wijken en Buurten data inlezen
cbs_wijkenbuurten_file = "CBS_wijkbuurtkaart/wijkenbuurten_2023_v1.gpkg"
cbs_gemeente = gpd.read_file(cbs_wijkenbuurten_file, layer="gemeenten")

cbs_gemeente = cbs_gemeente[['gemeentecode', 'gemeentenaam', 'geometry']]

#### Controleer data

In [ ]:
# print(cbs_gemeente.head())
print(cbs_gemeente.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   gemeentecode  424 non-null    object  
 1   gemeentenaam  424 non-null    object  
 2   geometry      424 non-null    geometry
dtypes: geometry(1), object(2)
memory usage: 10.1+ KB
None


In [ ]:
print(joined_data.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 8531 entries, 0 to 3962
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   POSTCODE4                   8531 non-null   int64   
 1   PRODUCTSOORT                8531 non-null   object  
 2   NETBEHEERDER                8531 non-null   object  
 3   NETGEBIED                   8531 non-null   object  
 4   WOONPLAATS                  8531 non-null   object  
 5   LANDCODE                    8531 non-null   object  
 6   VERBRUIKSSEGMENT            8531 non-null   object  
 7   AANSLUITINGEN_TOTAAL        8531 non-null   float64 
 8   SJV_GEMIDDELD_GEM           8531 non-null   float64 
 9   SJV_LAAG_TARIEF_PERC_GEM    8531 non-null   float64 
 10  LEVERINGSRICHTING_PERC_GEM  8531 non-null   float64 
 11  FYSIEKE_STATUS_PERC_GEM     8531 non-null   float64 
 12  SOORT_AANSLUITING_PERC_GEM  8531 non-null   float64 
 13  SLIMME_METER_PE

## Stap 7: Gemeente koppelen aan postcode 4 cijfers

In [ ]:
cbs_gemeente['geometry_right'] = cbs_gemeente.geometry

# Ruimtelijke join uitvoeren om de gemeentes te vinden die overlappen met postcodes
joined_data = gpd.sjoin(merged_data, cbs_gemeente, how='left', predicate='intersects')

# Bereken de overlappingsgebieden door een geometrische intersectie te maken tussen de postcodes en gemeentes
joined_data['intersection'] = joined_data.geometry.intersection(joined_data['geometry_right'])

# Bereken de oppervlakte van de intersectie
joined_data['intersection_area'] = joined_data['intersection'].area

# Groepeer per postcode en kies de gemeente met het grootste overlappingsgebied
idx = joined_data.groupby(
    ['POSTCODE4', 'PRODUCTSOORT', 'NETBEHEERDER', 'NETGEBIED', 'WOONPLAATS', 'LANDCODE', 'VERBRUIKSSEGMENT']
    )['intersection_area'].idxmax()

# Selecteer alleen de rijen met de grootste overlap per postcode
largest_overlap = joined_data.loc[idx]

# Rename geometry columns to have 'geometry' for the original geometry (geometry_left)
largest_overlap = largest_overlap.rename(columns={'geometry_left': 'geometry'})

# Behoud de gewenste kolommen
result = largest_overlap[['POSTCODE4', 'PRODUCTSOORT', 'NETBEHEERDER', 'NETGEBIED', 'WOONPLAATS',
                          'LANDCODE', 'VERBRUIKSSEGMENT', 'AANSLUITINGEN_TOTAAL', 'SJV_GEMIDDELD_GEM',
                          'SJV_LAAG_TARIEF_PERC_GEM', 'LEVERINGSRICHTING_PERC_GEM', 'FYSIEKE_STATUS_PERC_GEM',
                          'SOORT_AANSLUITING_PERC_GEM', 'SLIMME_METER_PERC_GEM', 'geometry',
                          'gemeentecode', 'gemeentenaam']]

#### Controleren

In [ ]:
# Print the updated merged_data
print(result.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 8531 entries, 0 to 3962
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   POSTCODE4                   8531 non-null   int64   
 1   PRODUCTSOORT                8531 non-null   object  
 2   NETBEHEERDER                8531 non-null   object  
 3   NETGEBIED                   8531 non-null   object  
 4   WOONPLAATS                  8531 non-null   object  
 5   LANDCODE                    8531 non-null   object  
 6   VERBRUIKSSEGMENT            8531 non-null   object  
 7   AANSLUITINGEN_TOTAAL        8531 non-null   float64 
 8   SJV_GEMIDDELD_GEM           8531 non-null   float64 
 9   SJV_LAAG_TARIEF_PERC_GEM    8531 non-null   float64 
 10  LEVERINGSRICHTING_PERC_GEM  8531 non-null   float64 
 11  FYSIEKE_STATUS_PERC_GEM     8531 non-null   float64 
 12  SOORT_AANSLUITING_PERC_GEM  8531 non-null   float64 
 13  SLIMME_METER_PE

## Stap 8: Elektrische data en gas data splitsen en gemeente filteren

In [ ]:
# Filter merged_data for Tilburg
filtered_data = result[(result['gemeentecode'] == 'GM0855')]

# Split into electricity and gas dataframes
electricity_data = filtered_data[filtered_data['PRODUCTSOORT'] == 'ELK']
gas_data = filtered_data[filtered_data['PRODUCTSOORT'] == 'GAS']

# Convert to GeoDataFrames
electricity_gdf = gpd.GeoDataFrame(electricity_data, geometry='geometry')
gas_gdf = gpd.GeoDataFrame(gas_data, geometry='geometry')


#### Controleer

In [ ]:
# Print the updated merged_data
print(electricity_gdf.info())
print(gas_gdf.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 57 entries, 382 to 668
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   POSTCODE4                   57 non-null     int64   
 1   PRODUCTSOORT                57 non-null     object  
 2   NETBEHEERDER                57 non-null     object  
 3   NETGEBIED                   57 non-null     object  
 4   WOONPLAATS                  57 non-null     object  
 5   LANDCODE                    57 non-null     object  
 6   VERBRUIKSSEGMENT            57 non-null     object  
 7   AANSLUITINGEN_TOTAAL        57 non-null     float64 
 8   SJV_GEMIDDELD_GEM           57 non-null     float64 
 9   SJV_LAAG_TARIEF_PERC_GEM    57 non-null     float64 
 10  LEVERINGSRICHTING_PERC_GEM  57 non-null     float64 
 11  FYSIEKE_STATUS_PERC_GEM     57 non-null     float64 
 12  SOORT_AANSLUITING_PERC_GEM  57 non-null     float64 
 13  SLIMME_METER_PER

## Stap 9: Geodata output naar gpkg bestand

In [ ]:
# Create a GeoPackage file
output_filename = 'netbeheerder_data.gpkg'

# Write the GeoDataFrames to the GeoPackage file
electricity_gdf.to_file(output_filename, layer='electricity', driver='GPKG')
gas_gdf.to_file(output_filename, layer='gas', driver='GPKG')

print(f"GeoPackage file '{output_filename}' created successfully.")

GeoPackage file 'netbeheerder_data.gpkg' created successfully.
